In [8]:
import sys

In [9]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [10]:
import numpy
import scipy
import pandas

In [11]:
import mysql_utils
import doc_proc
import init_tdm_tables

In [12]:
from importlib import reload

## Init Needed Tools

In [13]:
dd = lambda doc: doc_proc.build_text_feature(doc,
                                             components = ['title', 'summary'],
                                             lower=False,
                                             remove_stops=False,
                                             html_text=True,
                                            )

In [14]:
dp = init_tdm_tables.DocProcessor()

## Process

In [15]:
stops_lookup = mysql_utils.query_wordIDLookup(doc_proc.nltk_stops)

### Get Query Words

In [16]:
doc = {"title" : "FBI refused White House request to knock down recent Trump-Russia stories",
       "summary" : "Washington (CNN) The FBI rejected a recent White House request to publicly knock down media reports about communications between Donald Trump's associates and Russians known to US intelligence during the 2016 presidential campaign, multiple US officials briefed on the matter tell CNN."}

In [17]:
bow = dp.doc2BOW(dd(doc))

In [19]:
orig_query_words = [w for w in \
                    (mysql_utils.query_wordIDLookup([w for w in bow if w not in doc_proc.nltk_stops])).keys()
                   ]

### Query for Docs that Share Words

#### Orig Doc

In [20]:
word_count_info = mysql_utils.query_wordSummaryInfo(orig_query_words, wtype='id')

In [21]:
word_count_info.n_docs.describe()

count      31.000000
mean      723.741935
std      1476.303152
min        10.000000
25%        61.500000
50%       233.000000
75%       705.500000
max      7586.000000
Name: n_docs, dtype: float64

In [22]:
# Arbit...this should be "decayed"....see below
ndoc_cutoff = 100

qw_l01 = [int(w) for w in list(word_count_info[word_count_info.n_docs < ndoc_cutoff].word_id)]

In [23]:
# 2nd Pass: Get Doc IDs to use
docs_l01, wcs_l01 = mysql_utils.query_docsOnWords(qw_l01, word_type='id')

In [26]:
docs_l01

[3081,
 8219,
 11298,
 4131,
 11299,
 8230,
 17454,
 17455,
 15410,
 30771,
 1077,
 4150,
 30775,
 31798,
 6201,
 2105,
 3132,
 3136,
 8256,
 1090,
 8257,
 2116,
 30784,
 15429,
 7246,
 15438,
 1106,
 1107,
 2133,
 35925,
 15447,
 3160,
 31836,
 18533,
 7272,
 7281,
 115,
 30850,
 26759,
 1170,
 1179,
 5275,
 8351,
 3232,
 4256,
 5287,
 168,
 14507,
 30894,
 29870,
 184,
 15545,
 185,
 15552,
 30913,
 31938,
 5318,
 2249,
 15563,
 30928,
 4306,
 15578,
 4317,
 31968,
 13540,
 15589,
 31974,
 16616,
 31984,
 8436,
 6389,
 11513,
 8444,
 8446,
 7428,
 30982,
 32016,
 274,
 12571,
 31003,
 31005,
 27932,
 36127,
 11560,
 31017,
 3368,
 31019,
 27947,
 2344,
 18735,
 5429,
 30006,
 3389,
 2367,
 6469,
 326,
 3399,
 8517,
 15687,
 3398,
 7497,
 16716,
 11597,
 31052,
 2375,
 3408,
 15698,
 4436,
 29013,
 30038,
 8539,
 5468,
 16733,
 2409,
 3433,
 2412,
 4463,
 16762,
 3457,
 15746,
 32133,
 33164,
 19855,
 19856,
 32145,
 15765,
 31127,
 8604,
 16797,
 15773,
 34207,
 6557,
 32159,
 13730,

#### Level 1 Documents

### This stepo is a bottleneck; can i pre-reduce the query words by using info from the sub-set?

Have the word stats for all words queued up already..in the "words_info" table

In [28]:
query_words = mysql_utils.query_AllDocWords(docs_l01)
query_words = [w for w in query_words if w not in stops_lookup.keys()]
len(query_words)

4122

In [29]:
n_docs_tot = mysql_utils.query_totalEntries('rssfeed_links')
n_docs_tot

40787

In [30]:
bgwi = mysql_utils.query_backgroundWordInfo(query_words)
bgwi['frac'] = bgwi.n_docs / n_docs_tot

In [31]:
qwwi = mysql_utils.query_wordSummaryInfo(query_words, wtype='id', include_docs=docs_l01)
qwwi['frac'] = qwwi.n_docs / len(docs_l01)

In [32]:
wi = bgwi.join(qwwi, on=['word_id'], lsuffix='_bg', rsuffix='_qw')

In [33]:
wi.head()

,word_id_bg,n_docs_bg,n_total_bg,frac_bg,word_id_qw,n_docs_qw,n_total_qw,frac_qw
0,1,105,144,0.002574,3.0,80.0,123,0.233918
1,3,7586,11670,0.185991,11.0,6.0,6,0.017544
2,7,91,93,0.002231,15.0,6.0,10,0.017544
3,11,803,932,0.019688,19.0,1.0,1,0.002924
4,12,310,382,0.007600,20.0,44.0,60,0.128655


In [34]:
wi['ratio'] = wi.frac_qw / wi.frac_bg

In [35]:
wi.ratio.describe()

count    1697.000000
mean        7.171758
std        22.142781
min         0.024960
25%         0.779479
50%         1.780003
75%         4.417046
max       357.780702
Name: ratio, dtype: float64

In [36]:
# Restrict query words based on "words_info" table in mysql
# Say, more than 5 docs, but less than 100 (50) (20) docs?
word_filter_l02 = lambda x: (x['n_docs_qw'] >= 3 and x['n_docs_bg'] < 75 and x['ratio'] > 2)

In [37]:
ids2ints = lambda id_series: [int(i) for i in list(id_series)]

In [38]:
qw_l02 = ids2ints(wi.word_id_bg[wi.apply(lambda entry: word_filter_l02(entry), axis=1)])
print(len(qw_l02))

112


In [39]:
docs_l02, wcs_l02 = mysql_utils.query_docsOnWords(qw_l02,
                                      word_type="id", exclude_docs=docs_l01)

In [40]:
len(docs_l02)

3559

In [42]:
qww = mysql_utils.query_idWordLookup(qw_l02)

## Generate Network from Selected Vocab, Docs

In [43]:
words = set(orig_query_words).copy()
words.update(qw_l01)
words.update(qw_l02)
len(words)

142

In [44]:
for w in words:
    print(w)

3072
1024
1793
3
1539
1540
3075
775
3080
1040
19
2323
22
535
1046
1302
1561
1563
284
12578
290
292
1315
2853
1834
301
302
1325
304
1585
1586
5427
2100
2613
4148
311
56
1336
6202
826
3134
320
322
834
2116
27973
2863
1095
2632
1354
1355
2635
1869
2890
1618
339
2900
2901
2645
855
600
1625
2142
607
360
2153
1131
1648
113
2672
1398
121
2938
124
3453
1151
896
1153
898
902
1159
1670
2702
400
147
3220
410
155
156
411
412
668
1693
1694
1696
2208
166
423
1193
683
1196
3757
432
1968
688
1203
1716
2481
6327
1207
955
1723
701
1725
3262
1473
455
1484
1741
463
1234
211
979
1235
2775
218
987
1754
2268
222
990
2272
3034
2020
744
233
1001
1006
241
2294
2297
2300


In [45]:
orig_bow_lookup = {w : i for i,w in mysql_utils.query_wordIDLookup(bow.keys()).items()}

In [46]:
bow = {orig_bow_lookup[k] : wcount for k,wcount in bow.items()}

In [47]:
bows = [{'doc_id' : 'orig', 'bow' : bow}]
for d in docs_l01:
    bows.append({'doc_id' : d,
                 'bow' : mysql_utils.query_docBOW(d, word_list=words)})
for d in docs_l02:
    bows.append({'doc_id' : d,
                 'bow' : mysql_utils.query_docBOW(d, word_list=words)})
print(len(bows))

3902


In [47]:
import pickle

In [48]:
bows_path = os.path.join(os.path.abspath(os.path.join('../..')), 'src/data/sample_bows.pkl')

In [66]:
with open(bows_path, 'wb') as f:
    pickle.dump(bows, f)

In [1]:
from importlib import reload

In [ ]:
reload(mysql_ut)